In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import nltk
import re

In [3]:
df=pd.read_csv('train.tsv',sep='\t')

In [5]:
df.head(15)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [7]:
df.describe()

,PhraseId,SentenceId,Sentiment
count,156060.000000,156060.000000,156060.000000
mean,78030.500000,4079.732744,2.063578
std,45050.785842,2502.764394,0.893832
min,1.000000,1.000000,0.000000
25%,39015.750000,1861.750000,2.000000
50%,78030.500000,4017.000000,2.000000
75%,117045.250000,6244.000000,3.000000
max,156060.000000,8544.000000,4.000000


In [9]:
review=[]
for i in df['Sentiment']:
    if i==0:
        review.append('negative')
    elif i == 1:
        review.append('neutral')
    elif i == 2:
        review.append('somewhat negative')
    elif i == 3:
        review.append('somewhat positive')
    elif i == 4:
        review.append('positive')
    else:
        review.append('Failed')
df['Review'] = review

In [10]:
df['Review'].value_counts()

somewhat negative    79582
somewhat positive    32927
neutral              27273
positive              9206
negative              7072
Name: Review, dtype: int64

In [30]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [31]:
def text_process(reviewphrase):
    review=reviewphrase
    review = re.sub(r'[^a-zA-Z]',' ',str(review))
    review = review.lower()
    review = review.split()
    lemmatizer = WordNetLemmatizer()
    review = [lemmatizer.lemmatize(w) for w in review if not w in set(stopwords.words('english'))]
    return (' '.join(review))

In [33]:
processed_text= []
for i in range(0, 156060):
    
    processed_text.append(text_process(df['Phrase'][i]))

In [63]:
df['new_Phrase']=processed_text

In [75]:
positive=df[df['Review']=='positive']

In [76]:
p=positive['new_Phrase']

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

In [77]:
bow_transform=CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 2000)

In [78]:
pos_words = bow_transform.fit_transform(p)
pos_words = pos_words.toarray()
pos= bow_transform.get_feature_names()
dist = np.sum(pos_words, axis=0)


In [79]:
postive_new= pd.DataFrame(dist)
postive_new.columns=['word_count']
postive_new['word'] = pd.Series(pos, index=postive_new.index)



In [83]:
positivenew1=postive_new[['word','word_count']]
positivenew1.head()

,word,word_count
0,ability,20
1,able,32
2,absolute,9
3,absolutely,24
4,absorbing,18


In [70]:
df.drop(['Phrase'],axis=1,inplace=True)

In [84]:
df.head()

,PhraseId,SentenceId,Sentiment,Review,new_Phrase
0,1,1,1,neutral,series escapade demonstrating adage good goose...
1,2,1,2,somewhat negative,series escapade demonstrating adage good goose
2,3,1,2,somewhat negative,series
3,4,1,2,somewhat negative,
4,5,1,2,somewhat negative,series


In [85]:
cv = CountVectorizer(max_features = 1500)
X_train = cv.fit_transform(processed_text).toarray()
y=df.iloc[:,2].values

In [56]:
y=df.iloc[:,2]

In [86]:
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(X_train,y,test_size = 0.4,random_state = 100)

MNB = MultinomialNB()
MNB.fit(X_train, y_train)



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [87]:
y_pred=MNB.predict(X_test)

In [45]:
from sklearn.metrics import classification_report

In [88]:
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.48      0.13      0.21      2797
          1       0.48      0.18      0.27     10999
          2       0.60      0.91      0.72     31764
          3       0.52      0.30      0.38     13219
          4       0.54      0.18      0.27      3645

avg / total       0.55      0.58      0.52     62424

